# Imports

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm.auto import tqdm
from scipy import sparse
import torch

import warnings
warnings.filterwarnings('ignore')

# Data

## Load data

In [3]:
DATA_DIR = os.path.join("..", "data", "final_dataset")

In [35]:
interactions = pd.read_parquet(os.path.join(DATA_DIR, 'ratings.parquet'))
books= pd.read_parquet(os.path.join(DATA_DIR, 'books_all.parquet'))
users = pd.read_parquet(os.path.join(DATA_DIR, 'users.parquet'))
interactions = interactions[interactions["isbn"].isin(books["isbn"])]
interactions = interactions.query("provided_rating!=0")
interactions.reset_index(drop=True, inplace=True)
print(f"Number of ratings: {len(interactions)}")
print(f"Number of unique users: {interactions['user_id'].nunique()}")
print(f"Number of books: {interactions['isbn'].nunique()}")
interactions.head()

Number of ratings: 104756
Number of unique users: 31940
Number of books: 22020


,user_id,isbn,provided_rating
0,17,0891075275,6
1,17,0553264990,5
2,26,0449005615,9
3,39,0671888587,7
4,69,1853260053,8


In [29]:
books.head()

,num_pages,star_rating_1,star_rating_2,star_rating_3,star_rating_4,star_rating_5,average_rating,total_ratings,total_reviews,isbn,publication_date,title,series,book_number_in_series,authors,categories,format,lang
0,652,9896,25317,159960,556485,1546466,4.57,2298124,28062,None,2006-09-16,Harry Potter and the Half-Blood Prince,Harry Potter,6,[J. K. Rowling],"[Fantasy, Fantasy Books]",CD,en
1,870,12455,37005,211781,604283,1493113,4.50,2358637,29770,0439358078,2004-09-01,Harry Potter and the Order of the Phoenix,Harry Potter,5,[J.K. Rowling],"[Fantasy, Fantasy Books, Fantasy Books for Kids]",CD,en
2,309,108202,130310,567458,1513191,4268227,4.47,6587388,75911,None,2003-11-01,Harry Potter and the Sorcerer's Stone,Harry Potter,1,[J. K. Rowling],"[Thriller Books, Thrillers, Adventure Books, S...",Paperback,en
3,352,11896,49353,288821,706082,1504505,4.42,2560657,244,0439554896,2003-11-01,Harry Potter and the Chamber of Secrets,Harry Potter,2,[J.K. Rowling],"[Fantasy, Fantasy Books, Fantasy Books for Kids]",CD,en
4,435,10128,24849,194848,630534,1749958,4.57,2610317,37093,043965548X,2004-05-01,Harry Potter and the Prisoner of Azkaban,Harry Potter,3,[J.K. Rowling],"[Fantasy, Fantasy Books, Fantasy Books for Kids]",CD,en


In [36]:
users.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


## Preprocessing

As a part of preprocessing, we will do the following steps:
1. Create new columns, city, state and country from the location column.
2. Remove the location column.
3. Fill the age column with mean of the group of city, state and country. If some are remaining, use overall mean.
4. Select columns to be used from book details.

### Users Dataset

In [37]:
users["city"] = users["location"].str.split(", ").str[0]
users["state"] = users["location"].str.split(", ").str[1]
users["country"] = users["location"].str.split(", ").str[2]

In [48]:
groups = users.groupby(["city", "state", "country"]).groups
users_final = pd.DataFrame()
for group in tqdm(groups):
    temp_df = users.loc[groups[group]]
    if len(temp_df["age"].notnull()) == 0:
        users_final = pd.concat([users_final, temp_df], ignore_index=True)
    temp_df["age"].fillna(temp_df["age"].mean(), inplace=True)
    users_final = users_final.append(temp_df)


  0%|          | 0/57297 [00:00<?, ?it/s]

In [56]:
users_final["age"].isna().sum(), users["age"].isna().sum()

(18273, 110762)

In [57]:
users_final["age"].fillna(users_final["age"].mean(), inplace=True)

In [58]:
users_final["age"].isna().sum()

0

In [64]:
users_final = users_final[["user_id", "age", "city", "state", "country"]]
users_final["age"] = users_final["age"].astype(int)
users_final.sort_values(by="user_id", inplace=True)
users_final.reset_index(drop=True, inplace=True)
users_final.head()

,user_id,age,city,state,country
0,1,31,nyc,new york,usa
1,2,18,stockton,california,usa
2,3,55,moscow,yukon territory,russia
3,4,17,porto,v.n.gaia,portugal
4,5,35,farnborough,hants,united kingdom


### The Books Dataset

In [67]:
books.head().T

,0,1,2,3,4
num_pages,652,870,309,352,435
star_rating_1,9896,12455,108202,11896,10128
star_rating_2,25317,37005,130310,49353,24849
star_rating_3,159960,211781,567458,288821,194848
star_rating_4,556485,604283,1513191,706082,630534
star_rating_5,1546466,1493113,4268227,1504505,1749958
average_rating,4.57,4.5,4.47,4.42,4.57
total_ratings,2298124,2358637,6587388,2560657,2610317
total_reviews,28062,29770,75911,244,37093
isbn,None,0439358078,None,0439554896,043965548X


In [66]:
books.columns

Index(['num_pages', 'star_rating_1', 'star_rating_2', 'star_rating_3',
       'star_rating_4', 'star_rating_5', 'average_rating', 'total_ratings',
       'total_reviews', 'isbn', 'publication_date', 'title', 'series',
       'book_number_in_series', 'authors', 'categories', 'format', 'lang'],
      dtype='object')

## Smaller Dataset

In [5]:
num_ratings = df.groupby('isbn')['provided_rating'].count().sort_values(ascending=False)
most_rated_books = num_ratings.index[:10]
num_ratings = df.groupby('isbn')['provided_rating'].count().sort_values(ascending=False)
most_rated_books = num_ratings.index[:10]
num_ratings.head()

isbn
0316666343    707
0060928336    320
0671027360    269
067976402X    256
0786868716    242
Name: provided_rating, dtype: int64

In [6]:
df.groupby('user_id')['provided_rating'].count().sort_values(ascending=False)

user_id
11676     1593
98391      595
189835     371
76499      333
153662     322
          ... 
59675        1
157184       1
59685        1
59697        1
278854       1
Name: provided_rating, Length: 31940, dtype: int64

In [7]:
ratings = pd.DataFrame(df.groupby('isbn')['provided_rating'].mean())
ratings['num_ratings'] = pd.DataFrame(df.groupby('isbn')['provided_rating'].count())
ratings.head()

,provided_rating,num_ratings
isbn,,
0002163578,5.0,1
0002190915,9.5,2
0002210479,6.0,1
0002222469,8.0,1
0002241358,8.0,1


In [8]:
min_ratings = 5
books_ = ratings.query(f"num_ratings > {min_ratings}").index
print(f"Number of books_ with more than {min_ratings} ratings: {len(books_)}")
print(f"Original number of books_: {df['isbn'].nunique()}")
print(f"Number of rows in the original dataset: {df.shape[0]}")
df_small = df[df['isbn'].isin(books_)]
unique_users = df_small['user_id'].nunique()
print(f"Number of rows in the new dataset: {df_small.shape[0]}")
print(f"Number of unique users in the new dataset: {unique_users}")

Number of books_ with more than 5 ratings: 3823
Original number of books_: 22020
Number of rows in the original dataset: 104756
Number of rows in the new dataset: 72190
Number of unique users in the new dataset: 25812


## Preprocessing

In [9]:
n_users = df_small.user_id.nunique()
n_items = df_small.isbn.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+str(n_items))

Num. of Users: 25812
Num of Movies: 3823


In [10]:
user_id_map = dict(zip(df_small.user_id.unique(), list(range(n_users))))
book_id_map = dict(zip(df_small.isbn.unique(), list(range(n_items))))
user_id_map_df  = pd.DataFrame(
    {
        "user_id":user_id_map.keys(),
        "user_id_new": user_id_map.values(),
    }
)
book_id_map_df  = pd.DataFrame(
    {
        "isbn":book_id_map.keys(),
        "isbn_new": book_id_map.values(),
    }
)
df_small["user_id"] = df_small["user_id"].map(user_id_map)
df_small["isbn"] = df_small["isbn"].map(book_id_map)

In [28]:
df_small.head()

,user_id,isbn,provided_rating
0,0,0,6
1,0,1,5
2,1,2,9
3,2,3,7
4,3,4,8


In [11]:
def split_dataframe(df, holdout_fraction=0.1):
  """Splits a DataFrame into training and test sets.
  Args:
    df: a dataframe.
    holdout_fraction: fraction of dataframe rows to use in the test set.
  Returns:
    train: dataframe for training
    test: dataframe for testing
  """
  test = df.sample(frac=holdout_fraction, replace=False)
  train = df[~df.index.isin(test.index)]
  return train, test

train_df_, test_df_ = split_dataframe(df_small)

# Theory

Using neural networks instead of matrix factorization (MF) for recommendation systems has a number of advantages. See the notes section for detail. Here, we will be providing the model architecture that will be used for the recommendation system.

The model will consist of two steps:

## The Embedding Layer

This layer will take the user and item IDs in one-hot encoded form along with any other user and item feature and will pass it through a fully connected layer. The output of this layer will be the latent representation of the user and item. Let use denote $\mathbf{u}$ as the user and $\mathbf{v}$ as the item. Their dimensions will be $m+m_{uf}$ and $n+n_{if}$ respectively where $m$ and $n$ are the number of users and items and $m_{uf}$ and $n_{if}$ are the number of user and item features respectively. The output of the embedding layer will be $\mathbf{u} \in \mathbb{R}^d$ and $\mathbf{v} \in \mathbb{R}^d$ where $d$ is the dimension of the latent space.

We will have two different layers, one for users and the other of items. This is required because the number of users and items are different and we want to learn different embeddings for them. The user embedding layer will have $m+m_{uf}$ neurons and the item embedding layer will have $n+n_{if}$ neurons.

## CF Layers

CF layers, or collaborative filtering layers are made up of one or more layers of fully connected layers. The input to these layers will be the concatenation of the user and item latent representations. The output of the CF layers will be the predicted rating.

![](images/dl_01.png)

## Loss Function

The loss function can either be MSE or cross entropy. We will be experimenting with both.

## Data

We will be using all the positive data and a random sample of the negative data. The ratio of positive to negative data will be decided by a parameter.

# Model

In [27]:
train_df_.head()

,user_id,isbn,provided_rating
0,0,0,6
1,0,1,5
2,1,2,9
3,2,3,7
4,3,4,8
